In [1]:
import pandas as pd
import numpy as np
import altair as alt
import locale

locale.setlocale(locale.LC_ALL, 'sv_SE')
%config InlineBackend.figure_format = 'retina'

In [2]:
location="https://scb.se/hitta-statistik/statistik-efter-amne/befolkning/befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/"

source = pd.read_excel(location, sheet_name="Tabell 1", skiprows=5, header=1, index_col=0, usecols=[0,1,2,3,4,5,6])
source = source.stack().reset_index()
source['date'] = source['DagMånad'] + source['level_1'].apply(str)
source['date'] = pd.to_datetime(source['date'], format='%d %B%Y', errors='coerce')
source.rename(inplace=True, columns={'DagMånad': 'DayMonth', 'level_1': 'Year', 0: 'deaths'})
weekly = source.resample('W', on='date').sum().reset_index()

In [3]:
df_history = source[source['date'] < '2020-01-01']
df_current = source['2020-01-01' <= source['date']]
df_current = df_current[df_current['date'] < '2020-04-20']

history = alt.Chart(
    df_history,
    height=600,
    width=800,
    title="Daily Deaths in Sweden 2015-2020"
).mark_point(
    color='#d9d9d9'
).encode(
    x='monthdate(date)',
    y=alt.Y('deaths', title="Daily Deaths"),
)

mean = alt.Chart(df_history).mark_line(
    color='#969696',
    size=3
).transform_window(
    rolling_mean='mean(deaths)',
    frame=[-10, 10]
).encode(
    x='monthdate(date)',
    y=alt.Y('rolling_mean:Q'),
)

current = alt.Chart(df_current).mark_point(
    color='#fdae6b',
).encode(
    x='monthdate(date)',
    y='deaths',
)

current_mean = alt.Chart(df_current).mark_line(
    color='#e6550d',
    size=4
).transform_window(
    rolling_mean='mean(deaths)',
    frame=[-10, 10]
).encode(
    x='monthdate(date)',
    y=alt.Y('rolling_mean:Q'),
    tooltip=['deaths', 'rolling_mean:Q']
)

history + mean + current + current_mean.interactive()

alt.LayerChart(...)

In [4]:
df=weekly[weekly.deaths != 0]

last_date_to_use = str(df.date.max() - pd.Timedelta('7d'))
filter = f'year(datum.date) == 2020 & datetime(datum.date) < datetime("{last_date_to_use}")'

base = alt.Chart(df, title='Weekly Deaths in Sweden 2018-2020').mark_line(opacity=0.4).encode(
    x='monthdate(date)',
    y='sum(deaths)',
    color=alt.Color('year(date):N', scale=alt.Scale(scheme='category10'))
)

history = base.encode().transform_filter(
    "year(datum.date) < 2020"
)

current = base.encode().mark_line(opacity=1, color='blue').transform_filter(
    filter
)

history + current

alt.LayerChart(...)